In [1]:
import pandas as pd 


df = pd.read_csv("YouTube_Comments.csv")
df.head()

,Video ID,Comment,Author,Likes,Published At,Comment ID
0,EBOKYsWUhvI,Dub maro jaato 😢😢.\r\nTumse tumare ladkiya nhi...,HECTOR OF TROY,0.0,2023-07-06T07:04:02Z,UgwbyvIkkAhUdaCFpcp4AaABAg
1,EBOKYsWUhvI,Haar gye bechare,Geeta Saini,0.0,2023-07-03T22:21:37Z,Ugyz3OwSXamho91-8I94AaABAg
2,EBOKYsWUhvI,Dhamki mili pahalwano ko aur sab manage kr liy...,Ayaan Chouhan,0.0,2023-07-03T05:06:33Z,Ugyphs1TT1Yoj7MZBVJ4AaABAg
3,EBOKYsWUhvI,Jaato pr ye boj rhega ki vo apni hi vyavstha s...,Kamal,1.0,2023-07-02T03:39:50Z,UgwnIxpuFAcKsEzffBp4AaABAg
4,EBOKYsWUhvI,Pahlwan jante h kuch nhi kr payenge uska,jagriti tiwari upp,0.0,2023-06-28T15:10:36Z,UgxjuOlDigsmuDu53J54AaABAg


In [10]:
cleaned = df.drop(columns=["Video ID","Author","Likes","Published At","Comment ID"])

In [12]:
cleaned.head()

,Comment
0,Dub maro jaato 😢😢.\r\nTumse tumare ladkiya nhi...
1,Haar gye bechare
2,Dhamki mili pahalwano ko aur sab manage kr liy...
3,Jaato pr ye boj rhega ki vo apni hi vyavstha s...
4,Pahlwan jante h kuch nhi kr payenge uska


In [13]:
cleaned.to_csv("YouTube_Comments_Cleaned.csv")

In [25]:
import pandas as pd
import re

# Load dataset
file_path = "YouTube_Comments_Cleaned.csv"
df = pd.read_csv(file_path)

# Define common Hinglish words
common_hinglish_words = [
    "hai", "kya", "nahi", "bhai", "sab", "mat", "bata", "kar", "hai", "hona", "kr", "bht", "haan", "kyu"
]


In [28]:
# Function to detect Hinglish comments
# Function to detect Hinglish comments (without langdetect)
def is_hinglish(comment):
    comment = str(comment).lower()  # Convert to lowercase
    
    contains_latin = bool(re.search(r'[a-zA-Z]', comment))  # Must have Latin characters
    contains_devanagari = bool(re.search(r'[\u0900-\u097F]', comment))  # Should NOT have Hindi script
    contains_hinglish_word = any(word in comment for word in common_hinglish_words)  # Must have Hinglish word
    
    return contains_latin and not contains_devanagari and contains_hinglish_word

# Filter Hinglish comments
df_hinglish = df[df["Comment"].astype(str).apply(is_hinglish)].copy()

# Define refined hate speech categories
hate_categories = {
    "Religious Hate": ["mandir", "masjid", "hindu", "muslim", "allah", "bhagwan", "quran", "geeta"],
    "Caste-Based Hate": ["jaat", "brahmin", "dalit", "shudra", "baniya", "thakur"],
    "Gender-Based Hate": ["ladki", "ladka", "aurat", "mard", "mahila", "pati", "patni", "besharam"],
    "Political Hate": ["modi", "bjp", "congress", "rahul", "kejriwal", "sarkar", "andhbhakt", "pappu"],
    "Violent Speech": ["mar", "kat", "tod", "jal", "barbaad", "maar", "khoon", "chaku", "bandook"],
    "Abusive/Profane Speech": ["chutiya", "madarchod", "bhosdike", "gandu", "bhenchod", "harami"],
    "Ethnic/National Hate": ["pakistani", "bangladeshi", "bihari", "chinese", "kashmiri", "madrasi"]
}

# Function to classify hate speech
def classify_hate_speech(comment):
    comment_lower = comment.lower()
    for category, keywords in hate_categories.items():
        if any(keyword in comment_lower for keyword in keywords):
            return "Hate Speech", category
    return "Normal Text", "None"



In [29]:
# Apply hate speech classification
df_hinglish[["LABEL", "Category"]] = df_hinglish["Comment"].apply(lambda x: pd.Series(classify_hate_speech(x)))

# Rename column
df_hinglish = df_hinglish.rename(columns={"Comment": "Text"})

# Select relevant columns
df_final = df_hinglish[["Text", "LABEL", "Category"]]

# Display first few rows
df_final.head()


,Text,LABEL,Category
0,Dub maro jaato 😢😢.\r\nTumse tumare ladkiya nhi...,Hate Speech,Caste-Based Hate
2,Dhamki mili pahalwano ko aur sab manage kr liy...,Normal Text,None
3,Jaato pr ye boj rhega ki vo apni hi vyavstha s...,Hate Speech,Caste-Based Hate
4,Pahlwan jante h kuch nhi kr payenge uska,Normal Text,None
5,Logo ko ullu bana ke ye sab mahanubhav chal di...,Normal Text,None


In [30]:
df_final.to_csv("Hinglish_Hate_Speech_Labeled.csv", index=False)
print("Filtered and classified comments saved successfully.")

Filtered and classified comments saved successfully.


In [24]:
df.shape

(1021, 2)